In [ ]:
!pip install optuna
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from torch.utils.data import Dataset, DataLoader
import optuna

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 383.4/383.4 kB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.6/233.6 kB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 4.5 MB/s eta 0:00:00


In [ ]:
# Load and preprocess data
df = pd.read_csv("/content/drive/MyDrive/detect_dataset.csv")

In [ ]:
df

,Output (S),Ia,Ib,Ic,Va,Vb,Vc,Unnamed: 7,Unnamed: 8
0,0,-170.472196,9.219613,161.252583,0.054490,-0.659921,0.605431,NaN,NaN
1,0,-122.235754,6.168667,116.067087,0.102000,-0.628612,0.526202,NaN,NaN
2,0,-90.161474,3.813632,86.347841,0.141026,-0.605277,0.464251,NaN,NaN
3,0,-79.904916,2.398803,77.506112,0.156272,-0.602235,0.445963,NaN,NaN
4,0,-63.885255,0.590667,63.294587,0.180451,-0.591501,0.411050,NaN,NaN
...,...,...,...,...,...,...,...,...,...
11996,0,-66.237921,38.457041,24.912239,0.094421,-0.552019,0.457598,NaN,NaN
11997,0,-65.849493,37.465454,25.515675,0.103778,-0.555186,0.451407,NaN,NaN
11998,0,-65.446698,36.472055,26.106554,0.113107,-0.558211,0.445104,NaN,NaN
11999,0,-65.029633,35.477088,26.684731,0.122404,-0.561094,0.438690,NaN,NaN


In [ ]:
df.drop(columns=['Unnamed: 7', 'Unnamed: 8'], inplace=True)

In [ ]:
df.head()

,Output (S),Ia,Ib,Ic,Va,Vb,Vc
0,0,-170.472196,9.219613,161.252583,0.054490,-0.659921,0.605431
1,0,-122.235754,6.168667,116.067087,0.102000,-0.628612,0.526202
2,0,-90.161474,3.813632,86.347841,0.141026,-0.605277,0.464251
3,0,-79.904916,2.398803,77.506112,0.156272,-0.602235,0.445963
4,0,-63.885255,0.590667,63.294587,0.180451,-0.591501,0.411050


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(df.iloc[:, 1:], df.iloc[:, 0], test_size=0.2)

In [ ]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [ ]:
# Convert to tensors
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train.values, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test.values, dtype=torch.float32)


In [ ]:
# Dataset and DataLoader
class CustomDataset(Dataset):
    def __init__(self, features, labels):
        self.features = features
        self.labels = labels

    def __len__(self):
        return len(self.features)

    def __getitem__(self, idx):
        return self.features[idx], self.labels[idx]

In [ ]:
train_dataset = CustomDataset(X_train_tensor, y_train_tensor)
test_dataset = CustomDataset(X_test_tensor, y_test_tensor)

In [ ]:
# Define model architecture with dynamic hyperparameters
class NeuralNetwork(nn.Module):
    def __init__(self, input_size, hidden_layers, hidden_units):
        super(NeuralNetwork, self).__init__()
        layers = []
        in_size = input_size

        for _ in range(hidden_layers):
            layers.append(nn.Linear(in_size, hidden_units))
            layers.append(nn.ReLU())
            in_size = hidden_units

        layers.append(nn.Linear(in_size, 1))
        self.model = nn.Sequential(*layers)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        return self.sigmoid(self.model(x))

In [ ]:
# Optuna objective function
def objective(trial):
    # Suggest hyperparameters
    hyperparams = {
        'lr': trial.suggest_float('lr', 1e-5, 1e-2, log=True),
        'batch_size': trial.suggest_categorical('batch_size', [64, 128, 256]),
        'hidden_layers': trial.suggest_int('hidden_layers', 1, 3),
        'hidden_units': trial.suggest_categorical('hidden_units', [32, 64, 128]),
        # 'dropout_rate': trial.suggest_float('dropout_rate', 0.1, 0.5),
        'optimizer': trial.suggest_categorical('optimizer', ['Adam', 'SGD', 'RMSprop'])
    }

    # Create DataLoaders with current batch_size
    train_loader = DataLoader(train_dataset, batch_size=hyperparams['batch_size'], shuffle=True)
    test_loader = DataLoader(test_dataset, batch_size=hyperparams['batch_size'], shuffle=False)

    # Initialize model
    model = NeuralNetwork(
        input_size=X_train.shape[1],
        hidden_layers=hyperparams['hidden_layers'],
        hidden_units=hyperparams['hidden_units']
        # dropout_rate=hyperparams['dropout_rate']
    )

    # Select optimizer
    if hyperparams['optimizer'] == 'Adam':
        optimizer = torch.optim.Adam(model.parameters(), lr=hyperparams['lr'])
    elif hyperparams['optimizer'] == 'SGD':
        optimizer = torch.optim.SGD(model.parameters(), lr=hyperparams['lr'])
    else:
        optimizer = torch.optim.RMSprop(model.parameters(), lr=hyperparams['lr'])

    criterion = nn.BCELoss()

    # Training loop
    for epoch in range(50):  # Reduced epochs for faster tuning
        model.train()
        for features, labels in train_loader:
            optimizer.zero_grad()
            outputs = model(features)
            loss = criterion(outputs, labels.view(-1, 1))
            loss.backward()
            optimizer.step()

    # Evaluation
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for features, labels in test_loader:
            outputs = model(features)
            predicted = (outputs > 0.5).float()
            total += labels.size(0)
            correct += (predicted.view(-1) == labels).sum().item()

    accuracy = correct / total
    return accuracy

In [ ]:
# Run Optuna study
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=30)  # Reduce trials for quick testing


[I 2025-01-25 12:07:55,418] A new study created in memory with name: no-name-71742c23-6d78-43f8-bc95-ad63d63fded7
[I 2025-01-25 12:08:16,363] Trial 0 finished with value: 0.9950020824656394 and parameters: {'lr': 0.0014064737377190886, 'batch_size': 256, 'hidden_layers': 3, 'hidden_units': 64, 'optimizer': 'RMSprop'}. Best is trial 0 with value: 0.9950020824656394.
[I 2025-01-25 12:08:24,990] Trial 1 finished with value: 0.9970845481049563 and parameters: {'lr': 0.0014319469102240117, 'batch_size': 256, 'hidden_layers': 3, 'hidden_units': 128, 'optimizer': 'RMSprop'}. Best is trial 1 with value: 0.9970845481049563.
[I 2025-01-25 12:08:30,040] Trial 2 finished with value: 0.8446480633069554 and parameters: {'lr': 2.157451266497361e-05, 'batch_size': 256, 'hidden_layers': 1, 'hidden_units': 64, 'optimizer': 'RMSprop'}. Best is trial 1 with value: 0.9970845481049563.
[I 2025-01-25 12:08:36,923] Trial 3 finished with value: 0.6064139941690962 and parameters: {'lr': 0.00019531940909672506, 

In [ ]:
# Show best parameters
print("Best trial:")
trial = study.best_trial
print(f"  Value (Accuracy): {trial.value:.4f}")
print("  Params: ")
for key, value in trial.params.items():
    print(f"    {key}: {value}")

Best trial:
  Value (Accuracy): 0.9975
  Params: 
    lr: 0.0017701902717880862
    batch_size: 64
    hidden_layers: 3
    hidden_units: 128
    optimizer: RMSprop


In [ ]:
# Train final model with best parameters
best_params = trial.params
train_loader = DataLoader(train_dataset, batch_size=best_params['batch_size'], shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=best_params['batch_size'], shuffle=False)


In [ ]:
final_model = NeuralNetwork(
    input_size=X_train.shape[1],
    hidden_layers=best_params['hidden_layers'],
    hidden_units=best_params['hidden_units']
    # dropout_rate=best_params['dropout_rate']
)

if best_params['optimizer'] == 'Adam':
    optimizer = torch.optim.Adam(final_model.parameters(), lr=best_params['lr'])
elif best_params['optimizer'] == 'SGD':
    optimizer = torch.optim.SGD(final_model.parameters(), lr=best_params['lr'])
else:
    optimizer = torch.optim.RMSprop(final_model.parameters(), lr=best_params['lr'])

criterion = nn.BCELoss()

In [ ]:
# Full training with best parameters
for epoch in range(100):  # Train for more epochs
    final_model.train()
    for features, labels in train_loader:
        optimizer.zero_grad()
        outputs = final_model(features)
        loss = criterion(outputs, labels.view(-1, 1))
        loss.backward()
        optimizer.step()

    print(f'Epoch [{epoch+1}/100], Loss: {loss.item():.4f}')

Epoch [1/100], Loss: 0.0000
Epoch [2/100], Loss: 0.0985
Epoch [3/100], Loss: 0.0063
Epoch [4/100], Loss: 0.0125
Epoch [5/100], Loss: 0.0055
Epoch [6/100], Loss: 0.0034
Epoch [7/100], Loss: 0.0001
Epoch [8/100], Loss: 0.0041
Epoch [9/100], Loss: 0.0000
Epoch [10/100], Loss: 0.0005
Epoch [11/100], Loss: 0.0071
Epoch [12/100], Loss: 0.0000
Epoch [13/100], Loss: 0.0002
Epoch [14/100], Loss: 0.0048
Epoch [15/100], Loss: 0.0034
Epoch [16/100], Loss: 0.0001
Epoch [17/100], Loss: 0.0004
Epoch [18/100], Loss: 0.0077
Epoch [19/100], Loss: 0.0013
Epoch [20/100], Loss: 0.0024
Epoch [21/100], Loss: 0.0150
Epoch [22/100], Loss: 0.0103
Epoch [23/100], Loss: 0.0074
Epoch [24/100], Loss: 0.0000
Epoch [25/100], Loss: 0.0026
Epoch [26/100], Loss: 0.0119
Epoch [27/100], Loss: 0.0014
Epoch [28/100], Loss: 0.0006
Epoch [29/100], Loss: 0.0121
Epoch [30/100], Loss: 0.0004
Epoch [31/100], Loss: 0.0176
Epoch [32/100], Loss: 0.0001
Epoch [33/100], Loss: 0.0002
Epoch [34/100], Loss: 0.0002
Epoch [35/100], Loss: 0

In [ ]:
# Final evaluation
final_model.eval()
correct = 0
total = 0
with torch.no_grad():
    for features, labels in test_loader:
        outputs = final_model(features)
        predicted = (outputs > 0.5).float()
        total += labels.size(0)
        correct += (predicted.view(-1) == labels).sum().item()

print(f'Final Test Accuracy: {100 * correct / total:.2f}%')

Final Test Accuracy: 99.67%


In [ ]:
# Set the model to evaluation mode
final_model.eval()

# Lists to store predictions and true labels
all_predictions = []
all_labels = []

# Disable gradient computation for inference
with torch.no_grad():
    for features, labels in test_loader:
        # Forward pass: compute predictions
        outputs = final_model(features)

        # Apply sigmoid and threshold at 0.5 for binary classification
        predicted = (outputs > 0.5).float()

        # Store predictions and labels
        all_predictions.extend(predicted.view(-1).tolist())  # Flatten and convert to list
        all_labels.extend(labels.view(-1).tolist())  # Flatten and convert to list

# Convert lists to numpy arrays for easier comparison
all_predictions = np.array(all_predictions)
all_labels = np.array(all_labels)

# Print predicted and true outputs side by side
print("Predicted Outputs | True Outputs")
print("------------------|-------------")
for pred, true in zip(all_predictions, all_labels):
    print(f"{pred:17} | {true:11}")

Predicted Outputs | True Outputs
------------------|-------------
              0.0 |         0.0
              0.0 |         0.0
              1.0 |         1.0
              0.0 |         0.0
              0.0 |         0.0
              0.0 |         0.0
              0.0 |         0.0
              1.0 |         1.0
              0.0 |         0.0
              1.0 |         1.0
              1.0 |         1.0
              0.0 |         0.0
              0.0 |         0.0
              0.0 |         0.0
              1.0 |         1.0
              0.0 |         0.0
              0.0 |         0.0
              0.0 |         0.0
              1.0 |         1.0
              0.0 |         0.0
              1.0 |         1.0
              1.0 |         1.0
              0.0 |         0.0
              1.0 |         1.0
              0.0 |         0.0
              1.0 |         1.0
              0.0 |         0.0
              1.0 |         1.0
              0.0 |         0.0
      